# Przewidywanie bankructwa firm

Dominik Kędzierski, Krzysztof Wodnicki

[Zbiór danych](https://www.kaggle.com/datasets/fedesoriano/company-bankruptcy-prediction?resource=download)

## Import niezbędnych pakietów oraz zbioru danych

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import spearmanr
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from warnings import filterwarnings
from scipy.stats import randint, expon

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

%matplotlib inline
filterwarnings('ignore')

In [8]:
df = pd.read_csv("data.csv")

Z poprzedniej pracy domowej wiemy, że kolkumny `Liability-Assets Flag` i `Net Income Flag` nie wnoszą za dużo do naszego modelu, dlatego się ich pozbędziemy.

In [9]:
df = df.drop(" Liability-Assets Flag", axis = 1)
df = df.drop(" Net Income Flag", axis = 1)

### Podział zbioru danych

In [10]:
y = np.array(df["Bankrupt?"])
X = df.drop(["Bankrupt?"], axis = 1)

X_build, X_val, y_build, y_val = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=1
)

X_train, X_test, y_train, y_test = train_test_split(
    X_build, y_build, stratify=y_build, test_size=0.3, random_state=2
)

In [11]:
df_val = X_val.copy()
df_val["Bankrupt?"] = y_val.copy()
df_val.to_csv("data_val.csv")

df_train = X_train.copy()
df_train["Bankrupt?"] = y_train.copy()
df_train.to_csv("data_train.csv")

df_test = X_test.copy()
df_test["Bankrupt?"] = y_test.copy()
df_test.to_csv("data_test.csv")

## Preprocessing

Wykonujemy podobny preprocessing jak przy poprzedniej pracy domowej:
- zastąpienie wartości odstających (2.5%, 97.5%)
- zmiana kierunku zeminnych ujemnie skorelowanych ze zmienną celu
- normalizacja (min-max)

In [15]:
class Outliers(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X):
        self.max = {}
        self.min = {}
        for col in X.columns:
            self.max[col] = X[col].quantile(0.975)
            self.min[col] = X[col].quantile(0.025)
        return self
    
    def transform(self, X):
        for col in X.columns:
            if col != 'Bankrupt?':
                X[col] = np.where(X[col] < self.max[col], X[col], self.max[col])
                X[col] = np.where(X[col] > self.min[col], X[col], self.min[col])
        
        return X

In [16]:
class Direction(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X):
        self.reverse = set()
        tmp_y = X['Bankrupt?']
        for col in X.columns:
            if spearmanr(X[col], tmp_y)[0] < 0:
                    self.reverse.add(col)
        return self
    
    def transform(self, X):
        for col in X.columns:
            if col in self.reverse:
                X[col] = -X[col]
        return X

In [17]:
class Normalization(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X):
        self.minmaxscaler = MinMaxScaler().fit(X)
        return self
    
    def transform(self, X):
        X[X.columns] = MinMaxScaler().fit_transform(X)
        return X

In [18]:
preprocessing = Pipeline([
    ('outliers', Outliers()),
    ('direction', Direction()),
    ('normalization', Normalization())
])

In [19]:
df_train = pd.read_csv('data_train.csv', index_col=0)
df_train = preprocessing.fit_transform(df_train)

y_train = df_train['Bankrupt?']
X_train = df_train.drop('Bankrupt?', axis=1)

## Modele

Ze względu na mały udział bankrutów w naszym zbiorze danych, wybierając model, prawdopodobnie nie uda nam się stworzyć modelu, który będzie mieć na raz wysokie `accuracy`, `recall` i `precision`. Jednak w naszym problemie najważniejsze jest dla nas znalezienie wszytskich bankrutów (jeżeli zainwestujemy w firmę która zaraz zbankrutuje to prawdopodobnie stracimy więcej niż jeśli nie zainwestujemy wcale), dlatego priorytetem będzie uzyskanie jak najwyższej wartości `recall`.

### Decision Tree

In [22]:
dtc_clf = DecisionTreeClassifier(random_state=0)

dtc_gs_params = {
    'max_depth' : [5, 80, 150],
    'min_samples_leaf' : [1, 5, 25],
    'class_weight' : ['balanced', None],
    'max_features' : ['auto', 'sqrt', 'log2']
}

dtc_gs = GridSearchCV(estimator = dtc_clf,
                     param_grid = dtc_gs_params,
                     scoring = ['accuracy', 'precision', 'recall'],
                     refit = 'recall',
                     cv = 5,
                     verbose = 0)

dtc_rs_params = {
    'max_depth' : randint(3, 200),
    'min_samples_leaf' : randint(1, 30),
    'class_weight' : ['balanced', None],
    'max_features' : ['auto', 'sqrt', 'log2']
}

dtc_rs = RandomizedSearchCV(estimator = dtc_clf,
                     param_distributions = dtc_rs_params,
                     n_iter = 50,
                     scoring = ['accuracy', 'precision', 'recall'],
                     refit = 'recall',
                     cv = 5,
                     verbose = 0,
                     random_state = 0)

#### Grid search

In [23]:
dtc_gs.fit(X_train, y_train)
dtc_gs.best_params_

{'class_weight': 'balanced',
 'max_depth': 80,
 'max_features': 'auto',
 'min_samples_leaf': 25}

In [24]:
pd.DataFrame(dtc_gs.cv_results_).sort_values('rank_test_recall')[['mean_test_recall',
                                                                  'mean_test_precision',
                                                                  'mean_test_accuracy',
                                                                  'param_class_weight',
                                                                  'param_max_depth',
                                                                  'param_max_features',
                                                                  'param_min_samples_leaf'
                                                                 ]].head(5)

,mean_test_recall,mean_test_precision,mean_test_accuracy,param_class_weight,param_max_depth,param_max_features,param_min_samples_leaf
23,0.769264,0.160855,0.862620,balanced,150,sqrt,25
20,0.769264,0.160855,0.862620,balanced,150,auto,25
14,0.769264,0.160855,0.862620,balanced,80,sqrt,25
11,0.769264,0.160855,0.862620,balanced,80,auto,25
2,0.713853,0.174913,0.876975,balanced,5,auto,25


#### Random Search

In [28]:
dtc_rs.fit(X_train, y_train)
dtc_rs.best_params_

{'class_weight': 'balanced',
 'max_depth': 131,
 'max_features': 'auto',
 'min_samples_leaf': 22}

In [29]:
pd.DataFrame(dtc_rs.cv_results_).sort_values('rank_test_recall')[['mean_test_recall',
                                                                  'mean_test_precision',
                                                                  'mean_test_accuracy',
                                                                  'param_class_weight',
                                                                  'param_max_depth',
                                                                  'param_max_features',
                                                                  'param_min_samples_leaf'
                                                                 ]].head(5)

,mean_test_recall,mean_test_precision,mean_test_accuracy,param_class_weight,param_max_depth,param_max_features,param_min_samples_leaf
13,0.787013,0.164871,0.862616,balanced,131,auto,22
25,0.787013,0.164871,0.862616,balanced,26,sqrt,22
30,0.769697,0.167599,0.868604,balanced,45,sqrt,17
7,0.749351,0.171172,0.873993,balanced,82,auto,19
6,0.739827,0.169185,0.873993,balanced,12,auto,20


### Random Forest

In [30]:
rfc_clf = RandomForestClassifier(random_state=0)

rfc_gs_params = {
    'n_estimators' : [50, 100, 200],
    'max_depth' : [5, 20, 50],
    'class_weight' : ['balanced', 'balanced_subsample', None],
    'max_features' : ['auto', 'sqrt', 'log2']
}

rfc_gs = GridSearchCV(estimator = rfc_clf,
                     param_grid = rfc_gs_params,
                     scoring = ['accuracy', 'precision', 'recall'],
                     refit = 'recall',
                     cv = 4,
                     verbose = 0)

rfc_rs_params = {
    'n_estimators' : randint(50, 200),
    'max_depth' : randint(3, 50),
    'class_weight' : ['balanced', 'balanced_subsample', None],
    'max_features' : ['auto', 'sqrt', 'log2']
}

rfc_rs = RandomizedSearchCV(estimator = rfc_clf,
                     param_distributions = rfc_rs_params,
                     n_iter = 80,
                     scoring = ['accuracy', 'precision', 'recall'],
                     refit = 'recall',
                     cv = 4,
                     verbose = 0,
                     random_state = 0)

#### Grid search

In [31]:
rfc_gs.fit(X_train, y_train)
rfc_gs.best_params_

{'class_weight': 'balanced',
 'max_depth': 5,
 'max_features': 'log2',
 'n_estimators': 50}

In [32]:
pd.DataFrame(rfc_gs.cv_results_).sort_values('rank_test_recall')[['mean_test_recall',
                                                                  'mean_test_precision',
                                                                  'mean_test_accuracy',
                                                                  'param_class_weight',
                                                                  'param_max_depth',
                                                                  'param_max_features',
                                                                  'param_n_estimators'
                                                                 ]].head(5)

,mean_test_recall,mean_test_precision,mean_test_accuracy,param_class_weight,param_max_depth,param_max_features,param_n_estimators
6,0.657407,0.324251,0.943426,balanced,5,log2,50
7,0.638889,0.327466,0.944624,balanced,5,log2,100
34,0.638889,0.328622,0.944623,balanced_subsample,5,log2,100
35,0.638889,0.334743,0.946420,balanced_subsample,5,log2,200
8,0.629630,0.332602,0.946120,balanced,5,log2,200


#### Random Search

In [33]:
rfc_rs.fit(X_train, y_train)
rfc_rs.best_params_

{'class_weight': 'balanced',
 'max_depth': 3,
 'max_features': 'sqrt',
 'n_estimators': 71}

In [34]:
pd.DataFrame(rfc_rs.cv_results_).sort_values('rank_test_recall')[['mean_test_recall',
                                                                  'mean_test_precision',
                                                                  'mean_test_accuracy',
                                                                  'param_class_weight',
                                                                  'param_max_depth',
                                                                  'param_max_features',
                                                                  'param_n_estimators'
                                                                 ]].head(5)

,mean_test_recall,mean_test_precision,mean_test_accuracy,param_class_weight,param_max_depth,param_max_features,param_n_estimators
0,0.768519,0.256130,0.919481,balanced,3,sqrt,71
79,0.768519,0.251145,0.917986,balanced_subsample,3,auto,75
2,0.712963,0.282722,0.930856,balanced,4,log2,89
62,0.712963,0.303158,0.935942,balanced_subsample,4,sqrt,162
12,0.703704,0.296714,0.935645,balanced,4,sqrt,107


### Support Vector Machines

In [35]:
svm_clf = SVC(random_state=0)

svm_gs_params = {
    'class_weight' : ['balanced', None],
    'kernel' : ['poly', 'rbf', 'sigmoid'],
    'gamma' : [0.1, 1, 10],
    'C' : [0.1, 1, 10]
}

svm_gs = GridSearchCV(estimator = svm_clf,
                     param_grid = svm_gs_params,
                     scoring = ['accuracy', 'precision', 'recall'],
                     refit = 'recall',
                     cv = 5,
                     verbose = 0)

svm_rs_params = {
    'class_weight' : ['balanced', None],
    'kernel' : ['poly', 'rbf', 'sigmoid'],
    'gamma' : expon(),
    'C' : expon()
}

svm_rs = RandomizedSearchCV(estimator = svm_clf,
                     param_distributions = svm_rs_params,
                     n_iter = 50,
                     scoring = ['accuracy', 'precision', 'recall'],
                     refit = 'recall',
                     cv = 5,
                     verbose = 0,
                     random_state = 0)

#### Grid search

In [36]:
svm_gs.fit(X_train, y_train)
svm_gs.best_params_

{'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'}

In [37]:
pd.DataFrame(svm_gs.cv_results_).sort_values('rank_test_recall')[['mean_test_recall',
                                                                  'mean_test_precision',
                                                                  'mean_test_accuracy',
                                                                  'param_class_weight',
                                                                  'param_kernel',
                                                                  'param_gamma',
                                                                  'param_C'
                                                                 ]].head(5)

,mean_test_recall,mean_test_precision,mean_test_accuracy,param_class_weight,param_kernel,param_gamma,param_C
1,0.880087,0.193514,0.875480,balanced,rbf,0.1,0.1
44,0.800000,0.025749,0.219172,balanced,sigmoid,10,10
41,0.800000,0.025749,0.219172,balanced,sigmoid,1,10
19,0.676623,0.262286,0.925469,balanced,rbf,0.1,1
0,0.602165,0.226786,0.918288,balanced,poly,0.1,0.1


#### Random Search

In [38]:
svm_rs.fit(X_train, y_train)
svm_rs.best_params_

{'C': 0.1759135534346029,
 'class_weight': 'balanced',
 'gamma': 0.871915847002657,
 'kernel': 'sigmoid'}

In [39]:
pd.DataFrame(svm_rs.cv_results_).sort_values('rank_test_recall')[['mean_test_recall',
                                                                  'mean_test_precision',
                                                                  'mean_test_accuracy',
                                                                  'param_class_weight',
                                                                  'param_kernel',
                                                                  'param_gamma',
                                                                  'param_C'
                                                                 ]].head(5)

,mean_test_recall,mean_test_precision,mean_test_accuracy,param_class_weight,param_kernel,param_gamma,param_C
23,0.800000,0.025749,0.219172,balanced,sigmoid,0.582878,1.344245
17,0.800000,0.025749,0.219172,balanced,sigmoid,0.871916,0.175914
21,0.685714,0.263236,0.925169,balanced,rbf,0.101035,0.975102
19,0.556277,0.289492,0.940134,balanced,rbf,0.148712,1.123168
30,0.547186,0.296466,0.941931,balanced,rbf,0.180252,0.883129


## Podsumowanie

Analizując wyniki uzyskane przez różne modele, łatwo zauważyć, że jeśli zależy nam na wysokim wskaźniku `recall` to będziemy musieli pogodzić się z przynajmniej częściową utratą `accuracy`.

Przy dobieraniu parametrów dla modelu `Random Forest Classifier` niestety konieczne były pewne ogrnaniczenia, ze względu na potrzebną moc obliczeniową - bardzo długi czas obliczeń. Możliwe, że przy odpowiednio dobranych parametrach las losowy uzyskałby dużo lepsze wyniki.

Najlepsze wyniki udało się uzyskać korzystając z modelu `SVM` z parametrami:
- `C` = 0.1
- `class_weight` = 'balanced'
- `gamma` = 0.1
- `kernel` = 'rbf'

In [40]:
def results(X, y, model):
    accuracy = accuracy_score(y, model.predict(X))
    precision = precision_score(y, model.predict(X))
    recall = recall_score(y, model.predict(X))
    
    print('Accuracy: {:.2%}\nPrecision: {:.2%}\nRecall: {:.2%}'.format(accuracy, precision, recall))

In [41]:
model = svm_gs.best_estimator_

df_train = pd.read_csv('data_train.csv', index_col=0)
df_train = preprocessing.fit_transform(df_train)

y_train = df_train['Bankrupt?']
X_train = df_train.drop('Bankrupt?', axis=1)

df_test = pd.read_csv('data_test.csv', index_col=0)
df_test = preprocessing.transform(df_test)

y_test = df_test['Bankrupt?']
X_test = df_test.drop('Bankrupt?', axis=1)

In [42]:
results(X_train, y_train, model)

Accuracy: 87.91%
Precision: 20.28%
Recall: 93.52%


In [44]:
results(X_test, y_test, model)

Accuracy: 85.13%
Precision: 16.19%
Recall: 86.96%


Na zbiorze treningowym jest to aż 93% recall i 87% accuracy, natomiast na zbiorze testowym 86% recall i 85% accuracy.

## Walidacja

Walidujący: Jan Skwarek, Daniel Tytkowski

Zacznijmy od odtworzenia preprocessingu zaproponowanego przez budujących na zbiorze walidacyjnym.

In [45]:
df_val = pd.read_csv('data_val.csv', index_col=0)
df_val = preprocessing.transform(df_val)

y_val = df_val['Bankrupt?']
X_val = df_val.drop('Bankrupt?', axis=1)

Spójrzmy na rezultaty, jakie otrzymamy, przy zastosowaniu wybranych przez budujących, hiperparametrów.

In [46]:
results(X_val, y_val, model)

Accuracy: 87.68%
Precision: 18.58%
Recall: 83.33%


Co ciekawe, metryka `Recall`, która została wybrana przez budujących (zresztą słusznie) jako kluczowa, dała wynik o ponad 10% gorszy, niż w przypadku zbioru do treningu. Może to oznaczać, że model został przefitowany. Wniosek ten zresztą mógłby być już wyciągnięty przez samych budowniczych, ponieważ zbiór testowy też pokazywał znaczną różnicę. Tutaj różnica wynosi ponad 10%, a to już dużo. Co ciekawe dwie pozostałe metryki wypadają mniej więcej podobnie jak w przypadku zbioru treningowego i testowego. Należałoby się w takim razie zastanowić nad zastosowaniem standardowych procedur wykonywanych w przypadku overfittingu (regularyzacja i tym podobne).

Poza ogólnymi wnioskami napisanymi powyżej nie ma większych uwag do wykonanej pracy. Są za to oczywiście te mniejsze:
1. Brak komentarzy - zdecydowana większość pracy to nieprzerwany ciąg kodu (co prawda czytelnego, ale mogłyby się pojawić chociaż ostrzeżenia dla walidujących przed funkcjami wykonującymi się dłużej itp.).
2. Brak wyjaśnienia, dlaczego akurat te, a nie inne parametry były brane pod uwagę w `GridSearch` i `RandomizedSearch`.
3. Brak użycia wyszukiwania Bayes'a. Oczywiście nie można tego uznać za błąd, ponieważ w treści zadania było napisane jedynie o `GridSearch` oraz `RandomizedSearch`, aczkolwiek budujący mogliby się pokusić jeszcze o `BayesSearch`, który z całą pewnością spisałby się lepiej od `RandomizedSearch`.
4. Funkcje w preprocessingu mogłyby działać szybciej - zostać optymalniej i krócej napisane, przy pomocą na przykład metody `clip()`. Nie jest to też poważny problem bo wszystko działa jak ma działać.
5. Brak wizualizacji innych niż tabele - dla czytelniejszego porównania osiąganych wyników, budujący mogli się pokusić o jakieś wykresy lub inne tego typu wizualizacje.
6. Brak odniesienia się do paper'a podanego w treści zadania.
7. Budujący mogliby użyć większej ilości metryk do oceny modelu (chociażby ROC-AUC, gini).

Warto jednak dodać, że praca ta została wykonana bardzo dobrze. Nie mamy żadnych większych uwag, wszystko wydaje się być logiczne i czytelne.